<a href="https://colab.research.google.com/github/TonyQ2k3/pytorch-deeplearning/blob/main/fundamentals/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Pytorch

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [4]:
training_data = datasets.FashionMNIST(
  root='data',
  train=True,
  transform=ToTensor(),
  download=True
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    transform=ToTensor(),
    download=True
)

In [5]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )
  def forward(self, x):
    x_flattened = self.flatten(x)
    logits = self.stack(x_flattened)
    return logits

model = NeuralNetwork()

In [15]:
# Hyperparams
## Epochs: Number of times the dataset is passed through the model
## Batch size: Number of samples the model sees each epochs
## Step size (learning rate): Steps the model take to find a better weight. Too small => Take long time; Too large => May skip the best weight

learning_rate = 0.001
epochs = 5
batch_size = 64

# Loss func
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [19]:
# Training
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # Calculate loss
    y_preds = model(X)
    loss = loss_fn(y_preds, y)

    # Optimizing
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Testing
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      y_preds = model(X)
      test_loss += loss_fn(y_preds, y).item()
      correct += (y_preds.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= size
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
# Start training

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304463  [    0/60000]
loss: 2.287043  [ 6400/60000]
loss: 2.272563  [12800/60000]
loss: 2.264929  [19200/60000]
loss: 2.250368  [25600/60000]
loss: 2.242100  [32000/60000]
loss: 2.239611  [38400/60000]
loss: 2.223600  [44800/60000]
loss: 2.209448  [51200/60000]
loss: 2.190564  [57600/60000]
Test Error: 
 Accuracy: 57.7%, Avg loss: 0.034356 

Epoch 2
-------------------------------
loss: 2.207123  [    0/60000]
loss: 2.175255  [ 6400/60000]
loss: 2.148502  [12800/60000]
loss: 2.151077  [19200/60000]
loss: 2.095414  [25600/60000]
loss: 2.106207  [32000/60000]
loss: 2.101409  [38400/60000]
loss: 2.070063  [44800/60000]
loss: 2.051821  [51200/60000]
loss: 2.018984  [57600/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 0.031619 

Epoch 3
-------------------------------
loss: 2.057280  [    0/60000]
loss: 1.989863  [ 6400/60000]
loss: 1.940354  [12800/60000]
loss: 1.957582  [19200/60000]
loss: 1.843973  [25600/60000]
loss: 1.892722  [32000/600

In [21]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
